In [13]:
import pandas as pd
import datetime
from datetime import timedelta 
import import_ipynb
from utils_common import *

In [10]:
# Data of COVID-19
# source: DXY
_DXY_DATA_FILE_ = 'https://raw.githubusercontent.com/BlankerL/DXY-2019-nCoV-Data/master/csv/DXYArea.csv'
# source: JHU
_JHU_DATA_PATH_ = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/'
_JHU_DATA_START_DATE = '2020-01-22'
# source: Kaggle
_Kaggle_DATA_PATH_ = './data/data_kaggle.csv'

_Figure_PATH_ = './figures/'

In [11]:
# Some English names for provinces are not correct and need to be revised first
#chn_en = pd.read_csv(_CHN_EN_DICT_, encoding='utf-8')
#chn_en.loc[chn_en.Chinese == '贵州省','English'] = 'Guizhou'
#chn_en.loc[chn_en.Chinese == '海南省','English'] = 'Hainan'
#chn_en.loc[chn_en.Chinese == '内蒙古自治区','English'] = 'Inner Mongolia'
#chn_en.loc[chn_en.Chinese == '陕西省','English'] = 'Shaanxi'
#chn_en.to_csv('./data/locationDict.csv')

In [164]:
def rename_cities(snapshots):
    '''
    For now, 
    1. entries will be ignored if city_name == xxx(xxx), and xxx is already in the city_name set.
    2. entries will be renamed if city_name == xxx, where xxxy is already in the set with y being an administrative subdivision such as 县 and 区.
    '''
    
    dup_frm = snapshots[snapshots['city_name'].str.contains('（', na = False)]
    rename_dict = {}
    if len(dup_frm) >= 0:
        rename_dict = dict([(name, name.split('（')[0]) for name in dup_frm['city_name']])
    
    ###### 北京 Beijing
    rename_dict['大兴'] = '大兴区' # + 2020-01-31
    rename_dict['怀柔'] = '怀柔区' # + 
    rename_dict['昌平'] = '昌平区' # + 
    rename_dict['海淀'] = '海淀区' # +
    rename_dict['石景山'] = '石景山区'
    rename_dict['西城'] = '西城区'
    rename_dict['顺义'] = '顺义区'
    rename_dict['东城'] = '东城区'
    rename_dict['丰台'] = '丰台区'
    rename_dict['通州'] = '通州区'
    rename_dict['门头沟'] = '门头沟区'
    rename_dict['武汉来京人员'] = '外地来京人员' # + 2020-01-24
    #rename_dict['朝阳'] = '朝阳区' # + only for 北京市，not for 辽宁省
    
    ###### 重庆 Chongqing
    rename_dict['云阳'] = '云阳县' # * 2020-01-31 2020-01-24
    rename_dict['巫山'] = '巫山县' # * 2020-01-31 2020-01-24
    rename_dict['巫溪'] = '巫溪县' # * 2020-01-31 2020-01-24
    rename_dict['丰都'] = '丰都县'
    rename_dict['垫江'] = '垫江县'
    rename_dict['城口'] = '城口县'
    rename_dict['奉节'] = '奉节县'
    rename_dict['石柱'] = '石柱县'
    rename_dict['秀山'] = '秀山县'
    rename_dict['酉阳'] = '酉阳县'

    ###### 河南 Henan 
    ###### 邓州，永城，滑县，长垣县 to be reomved
    rename_dict['安阳市'] = '安阳' # + 2020-01-28 2020-02-01
    rename_dict['漯河市'] = '漯河'
    rename_dict['鹤壁市'] = '鹤壁'
    rename_dict['长垣'] = '长垣县' # + 2020-01-31 2020-02-01
    
    ###### 甘肃 Gansu
    rename_dict['天水市'] = '天水'
    rename_dict['平凉市'] = '平凉'
    rename_dict['白银市'] = '白银'
    rename_dict['金昌市'] = '金昌' # = 2020-01-28
    
    ###### 广东 Guangdong
    rename_dict['河源市'] = '河源' # + 2020-01-28
    rename_dict['外地来穗人员'] = '外地来粤人员' # = 2020-01-29
    
    ###### 海南 Hainan
    rename_dict['东方市'] = '东方' # + 2020-01-31
    rename_dict['琼海市'] = '琼海' # + 2020-01-31
    rename_dict['临高县'] = '临高' # + 2020-01-31
    rename_dict['澄迈县'] = '澄迈'
    rename_dict['琼中县'] = '琼中'
    rename_dict['陵水县'] = '陵水'
        
    ###### 河北 Hebei
    rename_dict['邯郸市'] = '邯郸' # + 2020-01-27 2020-01-28
    
    ###### 湖南 Hunan
    rename_dict['湘西自治州'] = '湘西州' # rename
    
    ###### 吉林 Jilin
    rename_dict['吉林市'] = '吉林'
    rename_dict['四平市'] = '四平' 
    
    ###### 内蒙古 Inner Mongolia
    rename_dict['乌海市'] = '乌海'
    rename_dict['锡林郭勒'] = '锡林郭勒盟' # simply rename it
    rename_dict['包头市东河区'] = '包头' # + 2020-01-27 ？######
    rename_dict['包头市昆都仑区'] = '包头' # + 2020-01-27 ？######
    rename_dict['赤峰市松山区'] = '赤峰' # * 2020-01-27
    rename_dict['赤峰市林西县'] = '赤峰' # * 2020-01-27
    rename_dict['通辽市经济开发区'] = '通辽'
    rename_dict['鄂尔多斯东胜区'] = '鄂尔多斯' # * 2020-01-27
    rename_dict['鄂尔多斯鄂托克前旗'] = '鄂尔多斯' # * 2020-01-27
    rename_dict['兴安盟乌兰浩特'] = '兴安盟' # + 2020-01-27
    rename_dict['呼伦贝尔满洲里'] = '呼伦贝尔' # * 2020-01-27
    rename_dict['满洲里'] = '呼伦贝尔' # 
    rename_dict['呼伦贝尔牙克石'] = '呼伦贝尔'
    rename_dict['呼伦贝尔牙克石市'] = '呼伦贝尔' # 满洲里 + 牙克石 = 呼伦贝尔
    rename_dict['锡林郭勒盟二连浩特'] = '锡林郭勒盟' # *
    rename_dict['锡林郭勒盟锡林浩特'] = '锡林郭勒盟' # *
    
    ###### 宁夏 Ningxia
    rename_dict['西宁市'] = '西宁' # 
    rename_dict['宁东管委会'] = '宁东' # *
    
    ###### 青海 Qinghai
    rename_dict['北海州'] = '海北州' # *
    
    ###### 山东 Shandong
    rename_dict['淄博市'] = '淄博' # + 2020-01-26
    
    ###### 山西 Shanxi
    rename_dict['临汾市'] = '临汾'
    rename_dict['朔州市'] = '朔州' # in
    
    ###### 上海 Shanghai
    rename_dict['杨浦'] = '杨浦区' # + 2020-01-31
    rename_dict['松江'] = '松江区' # + 2020-01-31
    rename_dict['金山'] = '金山区' # + 2020-01-31
    rename_dict['浦东'] = '浦东区'
    rename_dict['浦东新区'] = '浦东区'
    rename_dict['虹口'] = '虹口区'
    rename_dict['黄浦'] = '黄浦区'
    rename_dict['宝山'] = '宝山区'
    rename_dict['闵行'] = '闵行区'
    rename_dict['徐汇'] = '徐汇区'
    rename_dict['嘉定'] = '嘉定区'
    
    ###### 四川 Sichuan
    rename_dict['凉山'] = '凉山州' # + and rename
    #rename_dict['凉山州'] = '凉山彝族自治州' # + and rename
    
    ###### 天津 Tianjin
    rename_dict['外地来津'] = '外地来津人员' # + 2020-01-29
    
    ###### 新疆 Xinjiang
    rename_dict['吐鲁番市'] = '吐鲁番'   # raw data has both 吐鲁番市 and 吐鲁番, should be combined
    rename_dict['昌吉'] = '昌吉州' # + and rename
    rename_dict['第八师'] = '石河子'
    rename_dict['第八师石河子'] = '石河子'
    rename_dict['第八师石河子市'] = '石河子'
    rename_dict['兵团第八师石河子市'] = '石河子'
    rename_dict['兵团第五师五家渠市'] = '五家渠' # + and rename
    rename_dict['第六师'] = '五家渠' # + and rename
    rename_dict['兵团第六师五家渠市'] = '五家渠' # + and rename
    rename_dict['阿克苏地区'] = '阿克苏' # +
    rename_dict['第七师'] = '兵团第七师' # + 
    rename_dict['第九师'] = '兵团第九师'
    
    ###### 云南 Yunnan
    rename_dict['丽江市'] = '丽江' # + 
    rename_dict['大理'] = '大理州' # +? and rename 
    rename_dict['德宏'] = '德宏州' # +? and rename
    rename_dict['文山州'] = '文山' # + and rename 2020-02-10
    rename_dict['楚雄'] = '楚雄州' # + and rename 2020-02-10
    rename_dict['红河'] = '红河州' # + and rename 2020-02-10
    rename_dict['西双版纳'] = '西双版纳傣族自治州'
    rename_dict['西双版纳州'] = '西双版纳傣族自治州'
    
    ###### Unknown 
    ###### 待明确地区 未明确地区 未知 未知地区 待明确 未明确 不明地区
    rename_dict['待明确'] = '待明确地区' ######
    rename_dict['未明确'] = '待明确地区' 
    rename_dict['未明确地区'] = '待明确地区'
    rename_dict['未知'] = '待明确地区'
    rename_dict['未知地区'] = '待明确地区'
    rename_dict['不明地区'] = '待明确地区' ######

    snapshots['city_name'] = snapshots['city_name'].replace(rename_dict) # rename cities
    snapshots.loc[(snapshots['province_name'] == '北京市') & (snapshots['city_name'] == '朝阳'), 'city_name'] = '朝阳区' # only for 朝阳区 in 北京市
    return snapshots

# Calculate new confirmed, new dead, and new cured
def add_daily_new(df, group_keys=['province_name', 'city_name']):
    cols = ['confirmed', 'dead', 'cured']
    daily_new = df.groupby(group_keys).agg(dict([(n, 'diff') for n in ['cum_' + c for c in cols]]))
    daily_new = daily_new.rename(columns=dict([('cum_' + n, 'new_' + n) for n in cols]))
    df = pd.concat([df, daily_new], axis=1, join='outer')
    return df

In [157]:
# Data from JHU
def load_jhu_raw(verbose = False):
    date_list = pd.date_range(_JHS_DATA_START_DATE, datetime.date.today())
    frm_list = []
    for date in date_list:
        if verbose:
            print("Reading: " + str(date))
        try:
            frm = pd.read_csv(_JHS_DATA_PATH_ + date.strftime('%m-%d-%Y') + '.csv')
            frm_list.append(frm)
        except:
            continue
            
    out = pd.concat(frm_list, sort = False).drop_duplicates()

    rename_dict = {'Province/State': 'province/state', 
                   'Country/Region': 'country/region',
                   'Last Update': 'last_update',
                   'Confirmed': 'cum_confirmed',
                   'Deaths': 'cum_dead',
                   'Recovered': 'cum_cured',  
                   'Latitude': 'latitude',
                   'Longitude': 'longitude',
                  }

    out = out.rename(columns = rename_dict)

    out_others = out[~out['Country_Region'].isnull()]
    out_others = pd.concat([out_others.iloc[:,8:-1], out_others.iloc[:,3:6]], axis=1, sort=False)
    out = out[out['Country_Region'].isnull()].iloc[:,0:8]

    out['update_time'] = pd.to_datetime(out['last_update'])
    out['update_date'] = out['update_time'].dt.date
    out = out.drop(columns = ['last_update'])
    out = out.sort_values(by = ['update_time', 'country/region', 'province/state'])
    out = out.reset_index(drop = True)

    rename_dict = {'FIPS': 'fips', 
                   'Admin2': 'county',
                   'Province_State': 'province/state',
                   'Country_Region': 'country/region',
                   'Last_Update': 'last_update',
                   'Lat': 'latitude',
                   'Long_': 'longitude',
                   'Active': 'active'
                  }
    out_others = out_others.rename(columns = rename_dict)

    out_others['update_time'] = pd.to_datetime(out_others['last_update'])
    out_others['update_date'] = out_others['update_time'].dt.date
    out_others = out_others.drop(columns = ['last_update'])

    out_others_us = out_others[out_others['country/region'] =='US']
    out_others_us = out_others_us.sort_values(by = ['update_time', 'province/state', 'county'])
    out_others_us = out_others_us.reset_index(drop = True)

    out_others = out_others[out_others['country/region'] !='US'].iloc[:,2:]
    out_others = out_others.sort_values(by = ['update_time', 'country/region', 'province/state'])
    out_others = out_others.reset_index(drop = True)

    # only keep one piece of data every single day


    out.drop_duplicates(subset = ['update_date', 'province/state', 'country/region'], keep = 'last', inplace = True)
    out_others_us.drop_duplicates(subset = ['update_date', 'county', 'province/state'], keep = 'last', inplace = True) 
    out_others.drop_duplicates(subset = ['update_date', 'province/state', 'country/region'], keep = 'last', inplace = True) 


    out = out[['update_date', 'province/state', 'country/region', 'cum_confirmed', 'cum_dead', 'cum_cured', 'latitude', 'longitude']]
    out_others_us = out_others_us[['update_date', 'county', 'province/state', 'cum_confirmed', 'cum_dead', 'cum_cured', 'latitude', 'longitude']]
    out_others = out_others[['update_date', 'province/state', 'country/region', 'cum_confirmed', 'cum_dead', 'cum_cured', 'latitude', 'longitude']]

    return out, out_others_us, out_others

def jhu_daily(jhu_raw):
    frm_list = []
    for key, frm in jhu_raw.groupby([ 'update_date', 'province/state', 'country/region']):
        frm_list.append(frm[-1:])    
    out = pd.concat(frm_list).sort_values(['update_date',  'country/region', 'province/state'])
    out = add_daily_new(out, group_keys = ['country/region', 'province/state'])
    return out

In [199]:
out_province[out_province['province_name'] == '辽宁省'].head(5)

,province_name,province_zip_code,update_time,update_date,cum_confirmed,cum_cured,cum_dead
81319,辽宁省,210000,2020-01-22 16:47:22,2020-01-22,2,0,0
81289,辽宁省,210000,2020-01-23 11:55:16,2020-01-23,3,0,0
80346,辽宁省,210000,2020-01-24 23:35:03,2020-01-24,4,0,0
79336,辽宁省,210000,2020-01-25 19:17:40,2020-01-25,16,0,0
78581,辽宁省,210000,2020-01-26 16:50:30,2020-01-26,21,0,0


In [254]:
# Data from DXY
def load_chinese_raw():
    '''
    This provides a way to lookinto the 'raw' data
    '''
    raw = pd.read_csv(_DXY_DATA_FILE_)
    
    # the original CSV column names are in camel case, change to lower_case convention
    rename_dict = {'updateTime': 'update_time',
                   'continentName': 'continent_name',
                   'continentEnglishName': 'continent_name_en',
                   'countryName': 'country_name',
                   'countryEnglishName': 'country_name_en',
                   'provinceName': 'province_name',
                   'provinceEnglishName': 'province_name_en',
                   'province_zipCode': 'province_zip_code',
                   'cityName': 'city_name',
                   'cityEnglishName': 'city_name_en',
                   'city_zipCode': 'city_zip_code',
                   'province_confirmedCount': 'province_confirmed',
                   'province_suspectedCount': 'province_suspected',
                   'province_deadCount': 'province_dead',
                   'province_curedCount': 'province_cured',
                   'city_confirmedCount': 'city_confirmed',
                   'city_suspectedCount': 'city_suspected',
                   'city_deadCount': 'city_dead',
                   'city_curedCount': 'city_cured'
                  }
    data = raw.rename(columns=rename_dict)
    data['update_time'] = pd.to_datetime(data['update_time'])  # original type of update_time after read_csv is 'str'
    data['update_date'] = data['update_time'].dt.date   # add date for daily aggregation, if without to_datetime, it would be a dateInt object, difficult to use
    
    # display basic info
    print('Last update: ', data['update_time'].max())
    print('Data date range: ', data['update_date'].min(), 'to', data['update_date'].max())
    print('Number of rows in raw data: ', data.shape[0])
    return data

# Cleaning of data
# Not including Taiwan and Hongkong, including Macau and Tibet
def load_chinese_data():
    ''' This includes some basic cleaning'''
    data = load_chinese_raw()
    return rename_cities(data)

def dxy_daily(df):
    
    '''Aggregate the frequent time series data into a daily frame, ie, one entry per (date, province, city)'''
    
    df = df[df['country_name_en'] == 'China']
    df = df.drop(df.columns[0:4], axis = 1) # drop continent_name, continent_name_en, country_name, and country_name_en
    df = df[~df['province_name_en'].isin(['Hong Kong', 'Macau', 'Taiwan', 'Tibet', 'China'])]
    df = df.reset_index(drop = True)

    frm_list_city, frm_list_province = [], []
    to_names = ['confirmed', 'suspected', 'cured', 'dead']
    drop_cols = ['province_name_en', 'city_name_en']
    drop_province_cols = ['province_' + field for field in to_names] # put these in another dataframe
    province_cols = ['province_name', 'province_zip_code', 'update_time', 'update_date']

    # city-level data
    for key, frm in df.drop(columns = drop_cols + drop_province_cols).sort_values(['update_date']).groupby(['province_name', 'city_name', 'update_date']):
        frm_list_city.append(frm.sort_values(['update_time'])[-1:]) # take the latest row within (city, date)
    out_city = pd.concat(frm_list_city).sort_values(['update_date', 'province_name', 'city_name']) # convert a list to a dataframe

    out_city = out_city.rename(columns=dict([('city_' + d, 'cum_' + d) for d in to_names])) 
    out_city = out_city.drop(columns=['cum_suspected'])   # the suspected column from csv is not reliable, may keep this when the upstream problem is solved
    out_city = add_daily_new(out_city)  # add daily new cases
    out_city = add_en_location(out_city)

    # province-level data
    for key, frm in df[province_cols + drop_province_cols].sort_values(['update_date']).groupby(['province_name', 'update_date']):
        frm_list_province.append(frm.sort_values(['update_time'])[-1:]) # take the latest row within
    out_province = pd.concat(frm_list_province).sort_values(['update_date', 'province_name'])

    out_province = out_province.rename(columns=dict([('province_' + d, 'cum_' + d) for d in to_names])) 
    out_province = out_province.drop(columns=['cum_suspected'])

    # add missing data
    # source: http://www.nhc.gov.cn/xcs/yqtb/list_gzbd_2.shtml
    # （） （） （） （） （） （） # () stands for missing # * stands for being different
    out_province = out_province.append({'province_name' : '湖北省' , 'province_zip_code' : 420000, 'update_time': None, 'update_date': datetime.date(2020, 1, 15), 
                                        'cum_confirmed': 41, 'cum_cured': 7, 'cum_dead': 1} , ignore_index=True)
    out_province = out_province.append({'province_name' : '湖北省' , 'province_zip_code' : 420000, 'update_time': None, 'update_date': datetime.date(2020, 1, 16), 
                                        'cum_confirmed': 41, 'cum_cured': 12, 'cum_dead': 2} , ignore_index=True)
    out_province = out_province.append({'province_name' : '湖北省' , 'province_zip_code' : 420000, 'update_time': None, 'update_date': datetime.date(2020, 1, 17), 
                                        'cum_confirmed': 45, 'cum_cured': 15, 'cum_dead': 2} , ignore_index=True)
    out_province = out_province.append({'province_name' : '湖北省' , 'province_zip_code' : 420000, 'update_time': None, 'update_date': datetime.date(2020, 1, 18), 
                                        'cum_confirmed': 62, 'cum_cured': 19, 'cum_dead': 2} , ignore_index=True)
    out_province = out_province.append({'province_name' : '湖北省' , 'province_zip_code' : 420000, 'update_time': None, 'update_date': datetime.date(2020, 1, 19), 
                                        'cum_confirmed': 62 + 59, 'cum_cured': 19 + 5, 'cum_dead': 2 + 1} , ignore_index=True)
    out_province = out_province.append({'province_name' : '湖北省' , 'province_zip_code' : 420000, 'update_time': None, 'update_date': datetime.date(2020, 1, 20), 
                                        'cum_confirmed': 198, 'cum_cured': 25, 'cum_dead': 3} , ignore_index=True)
    # source: DXY
    # （）* （）
    out_province = out_province.append({'province_name' : '湖北省' , 'province_zip_code' : 420000, 'update_time': None, 'update_date': datetime.date(2020, 1, 21), 
                                        'cum_confirmed': 270, 'cum_cured': 25, 'cum_dead': 6} , ignore_index=True)
    out_province = out_province.append({'province_name' : '湖北省' , 'province_zip_code' : 420000, 'update_time': None, 'update_date': datetime.date(2020, 1, 22), 
                                        'cum_confirmed': 375, 'cum_cured': 28, 'cum_dead': 9} , ignore_index=True)
    out_province = out_province.append({'province_name' : '湖北省' , 'province_zip_code' : 420000, 'update_time': None, 'update_date': datetime.date(2020, 1, 23), 
                                        'cum_confirmed': 444, 'cum_cured': 28, 'cum_dead': 17} , ignore_index=True)
    # （） （）
    out_province = out_province.append({'province_name' : '广东省' , 'province_zip_code' : 440000, 'update_time': None, 'update_date': datetime.date(2020, 1, 20), 
                                        'cum_confirmed': 1, 'cum_cured': 0, 'cum_dead': 0} , ignore_index=True)
    out_province = out_province.append({'province_name' : '广东省' , 'province_zip_code' : 440000, 'update_time': None, 'update_date': datetime.date(2020, 1, 21), 
                                        'cum_confirmed': 17, 'cum_cured': 0, 'cum_dead': 0} , ignore_index=True)
    # （） （） * *
    out_province = out_province.append({'province_name' : '北京市' , 'province_zip_code' : 110000, 'update_time': None, 'update_date': datetime.date(2020, 1, 20), 
                                        'cum_confirmed': 5, 'cum_cured': 0, 'cum_dead': 0} , ignore_index=True)
    out_province = out_province.append({'province_name' : '北京市' , 'province_zip_code' : 110000, 'update_time': None, 'update_date': datetime.date(2020, 1, 21), 
                                        'cum_confirmed': 10, 'cum_cured': 0, 'cum_dead': 0} , ignore_index=True)
    out_province = out_province.append({'province_name' : '北京市' , 'province_zip_code' : 110000, 'update_time': None, 'update_date': datetime.date(2020, 1, 22), 
                                        'cum_confirmed': 14, 'cum_cured': 0, 'cum_dead': 0} , ignore_index=True)
    out_province = out_province.append({'province_name' : '北京市' , 'province_zip_code' : 110000, 'update_time': None, 'update_date': datetime.date(2020, 1, 23), 
                                        'cum_confirmed': 26, 'cum_cured': 0, 'cum_dead': 0} , ignore_index=True)
    # *
    out_province = out_province.append({'province_name' : '四川省' , 'province_zip_code' : 510000, 'update_time': None, 'update_date': datetime.date(2020, 1, 23), 
                                        'cum_confirmed': 7, 'cum_cured': 0, 'cum_dead': 0} , ignore_index=True)
    # * *
    out_province = out_province.append({'province_name' : '浙江省' , 'province_zip_code' : 330000, 'update_time': None, 'update_date': datetime.date(2020, 1, 22), 
                                        'cum_confirmed': 5, 'cum_cured': 0, 'cum_dead': 0} , ignore_index=True)
    out_province = out_province.append({'province_name' : '浙江省' , 'province_zip_code' : 330000, 'update_time': None, 'update_date': datetime.date(2020, 1, 23), 
                                        'cum_confirmed': 10, 'cum_cured': 0, 'cum_dead': 0} , ignore_index=True)
    # *
    out_province = out_province.append({'province_name' : '重庆市' , 'province_zip_code' : 500000, 'update_time': None, 'update_date': datetime.date(2020, 1, 22), 
                                        'cum_confirmed': 5, 'cum_cured': 0, 'cum_dead': 0} , ignore_index=True)
    # *
    out_province = out_province.append({'province_name' : '江苏省' , 'province_zip_code' : 320000, 'update_time': None, 'update_date': datetime.date(2020, 1, 23), 
                                        'cum_confirmed': 1, 'cum_cured': 0, 'cum_dead': 0} , ignore_index=True)
    # （）
    out_province = out_province.append({'province_name' : '上海市' , 'province_zip_code' : 310000, 'update_time': None, 'update_date': datetime.date(2020, 1, 21), 
                                        'cum_confirmed': 1, 'cum_cured': 0, 'cum_dead': 0} , ignore_index=True)
    # *
    out_province = out_province.append({'province_name' : '广西壮族自治区' , 'province_zip_code' : 450000, 'update_time': None, 'update_date': datetime.date(2020, 1, 23), 
                                        'cum_confirmed': 2, 'cum_cured': 0, 'cum_dead': 0} , ignore_index=True)
    # *
    out_province = out_province.append({'province_name' : '辽宁省' , 'province_zip_code' : 210000, 'update_time': None, 'update_date': datetime.date(2020, 1, 23), 
                                        'cum_confirmed': 2, 'cum_cured': 0, 'cum_dead': 0} , ignore_index=True)
    # * *
    out_province = out_province.append({'province_name' : '河南省' , 'province_zip_code' : 410000, 'update_time': None, 'update_date': datetime.date(2020, 1, 22), 
                                        'cum_confirmed': 1, 'cum_cured': 0, 'cum_dead': 0} , ignore_index=True)
    out_province = out_province.append({'province_name' : '河南省' , 'province_zip_code' : 410000, 'update_time': None, 'update_date': datetime.date(2020, 1, 23), 
                                        'cum_confirmed': 4, 'cum_cured': 0, 'cum_dead': 0} , ignore_index=True)
    # ()
    out_province = out_province.append({'province_name' : '河北省' , 'province_zip_code' : 130000, 'update_time': None, 'update_date': datetime.date(2020, 1, 23), 
                                        'cum_confirmed': 1, 'cum_cured': 0, 'cum_dead': 0} , ignore_index=True)
    # ()
    out_province = out_province.append({'province_name' : '山西省' , 'province_zip_code' : 140000, 'update_time': None, 'update_date': datetime.date(2020, 1, 23), 
                                        'cum_confirmed': 1, 'cum_cured': 0, 'cum_dead': 0} , ignore_index=True)
    # () ()
    out_province = out_province.append({'province_name' : '天津市' , 'province_zip_code' : 120000, 'update_time': None, 'update_date': datetime.date(2020, 1, 21), 
                                        'cum_confirmed': 2, 'cum_cured': 0, 'cum_dead': 0} , ignore_index=True)
    out_province = out_province.append({'province_name' : '天津市' , 'province_zip_code' : 120000, 'update_time': None, 'update_date': datetime.date(2020, 1, 23), 
                                        'cum_confirmed': 5, 'cum_cured': 0, 'cum_dead': 0} , ignore_index=True)
    # *
    out_province = out_province.append({'province_name' : '海南省' , 'province_zip_code' : 460000, 'update_time': None, 'update_date': datetime.date(2020, 1, 23), 
                                        'cum_confirmed': 4, 'cum_cured': 0, 'cum_dead': 0} , ignore_index=True)

    # *'s causes duplications
    out_province = out_province.drop_duplicates(subset = ['update_date', 'province_name']) 
    out_province = out_province.sort_values(['update_date', 'province_name'])
    out_province = out_province.reset_index(drop = True)

    ######################################## add missing data automatically (including provinces stopped being updated)

    names_province = sorted(set(out_province['province_name']))

    for name in names_province:
        df_single = out_province[out_province.province_name == name].copy()
        timespan = (max(df_single.update_date) - min(df_single.update_date)).days + 1
        date_last_local = max(df_single.update_date)
        date_last = max(out_province.update_date)
        if timespan != df_single.shape[0]: # missing data
            df_single['update_date'] =  pd.to_datetime(df_single['update_date'])
            df_single.set_index('update_date', inplace = True)
            df_single = df_single.resample('D').ffill().reset_index()
            df_single = df_single[out_province.columns]
            df_single['update_date'] = df_single['update_date'].dt.date
            out_province = out_province.append(df_single)
        if date_last_local < date_last: # if the province is stopped being updated
            df_single = df_single.append(df_single.iloc[-1])
            df_single.iloc[-1, df_single.columns.get_loc('update_date')] = date_last
            df_single['update_date'] =  pd.to_datetime(df_single['update_date'])
            df_single.set_index('update_date', inplace = True)
            df_single = df_single.resample('D').ffill().reset_index()
            df_single = df_single[out_province.columns]
            df_single['update_date'] = df_single['update_date'].dt.date
            out_province = out_province.append(df_single)

    out_province = out_province.drop_duplicates(subset = ['update_date', 'province_name']) 
    out_province = out_province.sort_values(['update_date', 'province_name'])
    out_province = out_province.reset_index(drop = True)
    ######################################## add missing data automatically

    out_province = add_daily_new(out_province, group_keys=['province_name'])  # add daily new cases
    out_province = add_en_location(out_province, tag = 'province')

    # rearrange columns
    new_col_order_city = ['update_date', 'province_name', 'province_name_en', 'province_zip_code', 'city_name', 'city_name_en', 'city_zip_code', 
                          'cum_confirmed', 'cum_cured', 'cum_dead', 'new_confirmed', 'new_cured', 'new_dead', 'update_time']
    if len(new_col_order_city) != len(out_city.columns):
        raise ValueError("Some columns are dropped: ", set(out_city.columns).difference(new_col_order_city))
    out_city = out_city[new_col_order_city]

    new_col_order_province = ['update_date', 'province_name', 'province_name_en', 'province_zip_code', 
                          'cum_confirmed', 'cum_cured', 'cum_dead', 'new_confirmed', 'new_cured', 'new_dead', 'update_time']
    if len(new_col_order_province) != len(out_province.columns):
        raise ValueError("Some columns are dropped: ", set(out_province.columns).difference(new_col_order_province))
    out_province = out_province[new_col_order_province]

    # convert from int to float
    out_city[['cum_confirmed', 'cum_cured', 'cum_dead']] = out_city[['cum_confirmed', 'cum_cured', 'cum_dead']].astype(float)
    out_province[['cum_confirmed', 'cum_cured', 'cum_dead']] = out_province[['cum_confirmed', 'cum_cured', 'cum_dead']].astype(float)

    # sort and reindex
    out_city = out_city.sort_values(['update_date', 'province_name_en', 'city_name_en'])
    out_city = out_city.reset_index(drop = True)

    out_province = out_province.sort_values(['update_date', 'province_name_en'])
    out_province = out_province.reset_index(drop = True)
    
    return out_city, out_province   

In [255]:
# data from Kaggle (out-dated)
def load_kaggle_raw():
    raw = pd.read_csv(_Kaggle_DATA_PATH_)
    out_country = raw[(raw.country == '中国') & (raw.province.isnull()) & (raw.city.isnull())]
    
    out_city = raw[(raw.country == '中国') & (raw.province.notnull()) & (raw.city.notnull())]
    out_city = out_city.drop(['country', 'countryCode'], axis=1)
    rename_dict = {'date': 'update_date',
                   'province': 'province_name',
                   'provinceCode': 'province_zip_code',
                   'city': 'city_name',
                   'cityCode': 'city_zip_code',
                   'confirmed': 'cum_confirmed',
                   'suspected': 'cum_suspected',
                   'cured': 'cum_cured',
                   'dead': 'cum_dead'
                      }
    out_city = out_city.rename(columns = rename_dict)
    out_city = add_en_location(out_city)
    out_city['update_date'] = pd.to_datetime(out_city['update_date'])  
    out_city['update_date'] = out_city['update_date'].dt.date
    out_city = out_city.sort_values(by = ['update_date', 'province_name', 'city_name'])
    out_city = out_city.reset_index(drop = True)
    new_cols = ['update_date', 'province_name', 'province_name_en', 'province_zip_code', 
                'city_name', 'city_name_en', 'city_zip_code',
                'cum_confirmed', 'cum_suspected', 'cum_cured', 'cum_dead']
    out_city = out_city[new_cols]
    
    out_province = raw[(raw.country == '中国') & (raw.province.notnull()) & (raw.city.isnull())]
    out_province = out_province.drop(['country', 'countryCode', 'city', 'cityCode'], axis=1)
    rename_dict = {'date': 'update_date',
                   'province': 'province_name',
                   'provinceCode': 'province_zip_code',
                   'confirmed': 'cum_confirmed',
                   'suspected': 'cum_suspected',
                   'cured': 'cum_cured',
                   'dead': 'cum_dead'
                      }
    out_province = out_province.rename(columns = rename_dict)
    out_province = add_en_location(out_province, tag = 'province')
    out_province['update_date'] = pd.to_datetime(out_province['update_date'])  
    out_province['update_date'] = out_province['update_date'].dt.date   
    out_province = out_province[~out_province.province_name.isin(['香港特别行政区','澳门特别行政区','台湾省'])]
    
    out_province = out_province.sort_values(by = ['update_date', 'province_name'])
    out_province = out_province.reset_index(drop = True)
    
    new_cols = ['update_date', 'province_name', 'province_name_en', 'province_zip_code', 
               'cum_confirmed', 'cum_suspected', 'cum_cured', 'cum_dead']
    out_province = out_province[new_cols]
    
    return out_city, out_province

In [160]:
# Data from JHU is incomplete !!!
data, data_others_us, data_others = load_jhu_raw()
data_JHU = jhu_daily(data)

In [222]:
data = load_chinese_data()

Last update:  2020-04-24 00:29:16
Data date range:  2020-01-22 to 2020-04-24
Number of rows in raw data:  148912


In [228]:
data_city, data_province = dxy_daily(data)

In [ ]:
# Compare different data sources
#data_JHU[data_JHU['province/state'] == 'Hubei']
#data_province[data_province['province_name_en'] == 'Hubei']

In [ ]:
# Save the data of Mainland China (national)
export_JHU = data_JHU[data_JHU['country/region'] == 'Mainland China'].groupby('update_date').agg('sum').drop(columns=['latitude','longitude'])
export_JHU.to_csv(r'./data/data_JHU.csv', index = False)
export_DXY_city = data_city.groupby('update_date').agg('sum').drop(columns=['province_zip_code', 'city_zip_code'])
export_DXY_city.to_csv(r'./data/data_DXY_city.csv', index = False)
export_DXY_province = data_province.groupby('update_date').agg('sum').drop(columns=['province_zip_code'])
export_DXY_province.to_csv(r'./data/data_DXY_province.csv', index = False)

In [256]:
# Save the data of Mainland China 
# city level
# province level

#data_city.to_csv(r'./data/data_DXY_city_all.csv', index = False)
#data_province.to_csv(r'./data/data_DXY_province_all.csv', index = False)

In [257]:
data_k_city, data_k_province = load_kaggle_raw()

In [ ]:
## Redundant names of cities

### 市 盟 州 县
#### 邯郸（市）乌海（市）河源（市）临汾（市）朔州（市）天水（市）平凉（市）
#### 白银（市）金昌（市）凉山州 东方（市）临高（县）澄迈（县）琼中（县）
#### 琼海（市） 陵水（县）
#### 杨浦区 松江区 金山区 浦东（新）区 
#### 丽江（市） 大理州 德宏州 文山（州）楚雄州 红河州 西双版纳州
#### 大兴区 怀柔区 昌平区 朝阳区 海淀区 石景山区 西城区 顺义区 东城区 丰台区 通州区 门头沟区
#### 西宁（市） 吉林（市） 四平（市）
#### 云阳县 巫山县 巫溪县 丰都县 垫江县 城口县 奉节县 石柱县 秀山县 酉阳县
#### 湘西（自治）州
#### 南阳（含邓州）商丘（含永城）安阳（市）  安阳（含滑县）新乡（含长垣）长垣县 漯河（市） 鹤壁（市）
#### 吐鲁番（市）昌吉州 第八师 = 第八师石河子 = 第八师石河子市 = 兵团第八师石河子市 = 石河子
#### 兵团第五师五家渠市 = 兵团第六师五家渠市 = 五家渠
#### 阿克苏（地区）
#### 淄博（市）
#### 锡林郭勒盟

### 内蒙古
#### 包头市（东河区）包头市（昆都仑区）呼和浩特（新城区）
#### 赤峰市（松山区）赤峰市（林西县）
#### 通辽市（经济开发区）
#### 鄂尔多斯（东胜区）鄂尔多斯（鄂托克前旗）
#### 兴安盟（乌兰浩特）
#### 呼伦贝尔（满洲里）呼伦贝尔（牙克石）呼伦贝尔（牙克石市）
#### 锡林郭勒盟（二连浩特）锡林郭勒盟（锡林浩特）

### 管委会
#### 宁东（管委会）

### 外地
#### 外地来穗人员 外地来粤人员 外地来京人员 武汉来京人员 外地来沪人员 外地来津人员（外地来津） 境外输入人员 

### 待明确地区
#### 待明确地区 未明确地区 未知 未知地区 待明确 未明确 不明地区

In [ ]:
#### 监狱
# 湖北 监狱系统
# 山东 任城监狱
# 浙江 省十里丰监狱
### 
# 沧州 is in 河北，instead of 河南

In [2]:
## Baidu Qianxi compared with DXY with respect to the naming of cities

### 安徽
#### 合肥 安庆 蚌埠 亳州 池州 滁州 阜阳 淮北 黄山 六安 马鞍山 宿州 铜陵 芜湖 宣城 淮南
##### 宿松（直管县）歙县（隶属黄山）

### 澳门
##### 澳门

### 北京
#### 

### 重庆

### 福建
#### 福州 龙岩 南平 宁德 莆田 泉州 三明 厦门 漳州

### 海南
#### 海口 白沙黎族自治县 保亭（黎族苗族自治县） 昌江（黎族自治县）
#### 儋州 澄迈 东方 定安 琼海 琼中（黎族苗族自治县） 乐东（黎族自治县）
#### 临高 陵水（黎族自治县） 三亚 屯昌 万宁 文昌 五指山 三沙

### 河北
#### 石家庄 保定 沧州 承德 邯郸 衡水 廊坊 秦皇岛 唐山 邢台 张家口

### 河南
#### 郑州 安阳 鹤壁 焦作 开封 洛阳 漯河 南阳 平顶山 濮阳 三门峡 商丘 新乡 信阳 许昌 周口 驻马店 济源
##### 固始县（直管县，隶属信阳）巩义（直管县，隶属郑州/洛阳）永城（商丘市代管县级市），
##### 滑县（直管县）邓州（直管县，隶属安阳）长垣县（直管县，隶属新乡）

### 黑龙江
#### 哈尔滨 大庆 大兴安岭（地区） 鹤岗 黑河 鸡西 佳木斯 牡丹江 七台河 
#### 齐齐哈尔 双鸭山 绥化 伊春

### 湖北
#### 武汉 鄂州 恩施（恩施州） 黄冈 黄石 荆门 荆州 潜江 神农架林区 十堰 随州 天门 仙桃 咸宁 襄阳 孝感 宜昌
#### 恩施土家族苗族自治州

### 湖南
#### 长沙 常德 郴州 衡阳 怀化 娄底 邵阳 湘潭 湘西州 益阳 永州 岳阳 张家界 株洲
#### 湘西土家族苗族自治州
 
### 甘肃
#### 兰州 白银 定西 甘南（州） 嘉峪关 金昌 酒泉 临夏（州） 陇南 平凉 庆阳 天水 武威 张掖
#### 临夏回族自治州 甘南藏族自治区

### 广东
#### 广州 潮州 东莞 佛山 河源 惠州 江门 揭阳 茂名 梅州 清远 汕头 汕尾 韶关 深圳 阳江 云浮 湛江 肇庆 中山 珠海

### 广西（壮族自治区）
#### 南宁 百色 北海 崇左 防城港 桂林 贵港 河池 贺州 来宾 柳州 钦州 梧州 玉林

### 贵州
#### 贵阳 安顺 毕节（地区） 六盘水 铜仁（地区） 遵义 黔西南州 黔东南州 黔南州
#### 黔东南苗族侗族自治州 黔南布依族苗族自治州 黔西南布依族苗族自治州

### 吉林
#### 长春 白城 白山 吉林市 辽源 四平 松原 通化 延边
#### 延边朝鲜族自治州
##### 公主岭（隶属四平）梅河口（隶属通化）

### 江苏
#### 南京 常州 淮安 连云港 南通 苏州 宿迁 泰州 无锡 徐州 盐城 扬州 镇江

### 江西
#### 南昌 抚州 赣州 吉安 景德镇 九江 萍乡 上饶 新余 宜春 鹰潭
##### 赣江新区（国家级新区，隶属南昌）

### 辽宁
#### 沈阳 鞍山 本溪 朝阳 大连 丹东 抚顺 阜新 葫芦岛 锦州 辽阳 盘锦 铁岭 营口

### 内蒙古（自治区）
#### 呼和浩特 阿拉善盟 包头 巴彦淖尔 赤峰 鄂尔多斯 呼伦贝尔 通辽 乌海 乌兰察布 锡林郭勒盟 兴安盟
#### 满洲里（隶属呼伦贝尔）

### 宁夏（回族自治区）
#### 银川 固原 石嘴山 吴忠 中卫
#### 宁东

### 青海
#### 西宁 果洛州 海东地区 海北州 海南州 海西州 黄南州 玉树州
#### 海北藏族自治州
#### 北海州（typo）

### 上海
#### 

### 陕西：
#### 西安 安康 宝鸡 汉中 商洛 铜川 渭南 咸阳 延安 榆林 
##### 杨凌（区，隶属咸阳）， 韩城（隶属渭南）

### 山东
#### 济南 滨州 东营 德州 菏泽 济宁 聊城 临沂 青岛 日照 泰安 威海 潍坊 烟台 枣庄 淄博

### 山西:	
#### 太原 长治 大同 晋城 晋中 临汾 吕梁 朔州 忻州 阳泉 运城

### 四川：
#### 成都 阿坝州 巴中 达州 德阳 甘孜州 广安 广元 乐山 凉山州
#### 泸州 南充 眉山 绵阳 内江 攀枝花 遂宁 雅安 宜宾 资阳 自贡
#### 阿坝藏族羌族自治州 凉山彝族自治州 甘孜藏族自治州

### 天津：
####

### 西藏：
#### 拉萨 阿里地区 昌都地区 林芝地区 那曲地区 日喀则地区 山南地区

### 新疆：
#### 乌鲁木齐 阿拉尔 阿克苏（地区） 阿勒泰地区 巴（音郭楞蒙古自治）州 博尔塔拉州 昌吉州
#### 哈密地区 和田地区 喀什地区 克拉玛依 克孜勒苏州 石河子 塔城（地区） 图木舒克 吐鲁番（地区） 
#### 五家渠 伊犁州 北屯 铁门关 双河 可克达拉 昆玉
#### 昌吉回族自治州
##### 兵团第四师 兵团第七师 兵团第九师 兵团第十二师 第六师 第七师 第九师 胡杨河

#### 云南
#### 昆明 保山 楚雄州 大理州 德宏州 迪庆州 红河州 丽江 临沧 怒江州 
#### 普洱 曲靖 昭通 文山 西双版纳傣族自治州 玉溪
#### 大理白族自治州 德宏傣族景颇族自治州 文山壮族苗族自治州 楚雄彝族自治州
#### 红河哈尼族彝族自治州 西双版纳傣族自治州

### 浙江：
#### 杭州 湖州 嘉兴 金华 丽水 宁波 衢州 绍兴 台州 温州 舟山

In [3]:
### Missing data added for provinces (cum_confirmed, cum_dead, cum_cured)
#### Hubei 01-21 to 01-23: 
# 270, 6, 25
# 375, 9, 28
# 444, 17, 28
#### Shandong (JHU) 01-22 to 01-23:
# 2, 0, 0
# 6, 0, 0
#### Guangdong 01-20 to 01-23:
# 1, 0, 0
# 17, 0, 0
# 26, 0, 0
# 32, 0, 0
#### Beijing 01-20 to 01-23
# 5, 0, 0
# 10, 0, 0
# 14, 0, 0
# 26, 0, 0
#### Sichuan 01-22 to 01-23
# 5, 0, 0 (JHU)
# 7, 0, 0
#### Heilongjiang
#### Zhejiang 01-22 to 01-23
# 5, 0, 0
# 10, 0, 0
#### Hunan
#### Chongqing 01-22 to 01-23
# 5, 0, 0
# 9, 0, 0
#### Hongkong 01-23
# 1, 0, 0
#### Jiangsu 01-23
# 1, 0, 0
#### Shanghai 01-21 to 01-26
# 1, 0, 0
# 9, 0, 0
# 16, 0, 0
# 20, 0, 0
# 33, 1, 0
# 40, 1, 1
#### Guangxi 01-23
# 2, 0, 0
#### Guizhou
#### Jiangxi 01-22 to 01-23
# 2, 0, 0
# 3, 0, 0
#### Shaanxi
#### Liaoning 01-23
# 2, 0, 0
#### Fujian (JHU) 01-22 to 01-23
# 1, 0, 0
# 5, 0, 0
#### Anhui 01-23
# 9, 0, 0
#### Gansu
#### Henan 01-22 to 01-23
# 1, 0, 0
# 4, 0, 0
#### Inner Mongolia (error)
#### Hebei 01-23
# 1, 0, 0
#### Shanxi 01-23
# 1, 0, 0
#### Ningxia
#### Tianjin 01-21 to 01-25
# 2, 0, 0
# 4, 0, 0
# 5, 0, 0
# 8, 0, 0
# 10, 0, 0
#### Hainan 01-23
# 4, 0, 0
#### Jilin
#### Xinjiang 01-24
# 2, 0, 0
#### Macau
#### Qinghai
#### Tibet

In [ ]:
#### missing data
#### data from DXY only got updated if there was new changes

#### extra column: 'province_suspected'

# front

    out_province = out_province.append({'province_name' : '山东省' , 'province_zip_code' : 370000, 'update_time': None, 'update_date': datetime.date(2020, 1, 22), 
                                        'province_confirmed': 2, 'province_suspected': None, 'province_cured': 0, 'province_dead': 0} , ignore_index=True)
    out_province = out_province.append({'province_name' : '山东省' , 'province_zip_code' : 370000, 'update_time': None, 'update_date': datetime.date(2020, 1, 23), 
                                        'province_confirmed': 6, 'province_suspected': None, 'province_cured': 0, 'province_dead': 0} , ignore_index=True)
    
    out_province = out_province.append({'province_name' : '广东省' , 'province_zip_code' : 440000, 'update_time': None, 'update_date': datetime.date(2020, 1, 22), 
                                        'province_confirmed': 26, 'province_suspected': None, 'province_cured': 0, 'province_dead': 0} , ignore_index=True)
    out_province = out_province.append({'province_name' : '广东省' , 'province_zip_code' : 440000, 'update_time': None, 'update_date': datetime.date(2020, 1, 23), 
                                        'province_confirmed': 32, 'province_suspected': None, 'province_cured': 0, 'province_dead': 0} , ignore_index=True)
    
    out_province = out_province.append({'province_name' : '四川省' , 'province_zip_code' : 510000, 'update_time': None, 'update_date': datetime.date(2020, 1, 22), 
                                        'province_confirmed': 5, 'province_suspected': None, 'province_cured': 0, 'province_dead': 0} , ignore_index=True)
    
    out_province = out_province.append({'province_name' : '重庆市' , 'province_zip_code' : 500000, 'update_time': None, 'update_date': datetime.date(2020, 1, 23), 
                                        'province_confirmed': 9, 'province_suspected': None, 'province_cured': 0, 'province_dead': 0} , ignore_index=True)
    
    out_province = out_province.append({'province_name' : '上海市' , 'province_zip_code' : 310000, 'update_time': None, 'update_date': datetime.date(2020, 1, 22), 
                                        'province_confirmed': 9, 'province_suspected': None, 'province_cured': 0, 'province_dead': 0} , ignore_index=True)
    out_province = out_province.append({'province_name' : '上海市' , 'province_zip_code' : 310000, 'update_time': None, 'update_date': datetime.date(2020, 1, 23), 
                                        'province_confirmed': 16, 'province_suspected': None, 'province_cured': 0, 'province_dead': 0} , ignore_index=True)
    out_province = out_province.append({'province_name' : '上海市' , 'province_zip_code' : 310000, 'update_time': None, 'update_date': datetime.date(2020, 1, 24), 
                                        'province_confirmed': 20, 'province_suspected': None, 'province_cured': 0, 'province_dead': 0} , ignore_index=True)
    out_province = out_province.append({'province_name' : '上海市' , 'province_zip_code' : 310000, 'update_time': None, 'update_date': datetime.date(2020, 1, 25), 
                                        'province_confirmed': 33, 'province_suspected': None, 'province_cured': 0, 'province_dead': 1} , ignore_index=True)
    out_province = out_province.append({'province_name' : '上海市' , 'province_zip_code' : 310000, 'update_time': None, 'update_date': datetime.date(2020, 1, 26), 
                                        'province_confirmed': 40, 'province_suspected': None, 'province_cured': 1, 'province_dead': 1} , ignore_index=True)
    
    out_province = out_province.append({'province_name' : '江西省' , 'province_zip_code' : 360000, 'update_time': None, 'update_date': datetime.date(2020, 1, 22), 
                                        'province_confirmed': 2, 'province_suspected': None, 'province_cured': 0, 'province_dead': 0} , ignore_index=True)
    out_province = out_province.append({'province_name' : '江西省' , 'province_zip_code' : 360000, 'update_time': None, 'update_date': datetime.date(2020, 1, 23), 
                                        'province_confirmed': 3, 'province_suspected': None, 'province_cured': 0, 'province_dead': 0} , ignore_index=True)
    
    out_province = out_province.append({'province_name' : '福建省' , 'province_zip_code' : 350000, 'update_time': None, 'update_date': datetime.date(2020, 1, 22), 
                                        'province_confirmed': 1, 'province_suspected': None, 'province_cured': 0, 'province_dead': 0} , ignore_index=True)
    out_province = out_province.append({'province_name' : '福建省' , 'province_zip_code' : 350000, 'update_time': None, 'update_date': datetime.date(2020, 1, 23), 
                                        'province_confirmed': 5, 'province_suspected': None, 'province_cured': 0, 'province_dead': 0} , ignore_index=True)


    out_province = out_province.append({'province_name' : '安徽省' , 'province_zip_code' : 340000, 'update_time': None, 'update_date': datetime.date(2020, 1, 23), 
                                        'province_confirmed': 9, 'province_suspected': None, 'province_cured': 0, 'province_dead': 0} , ignore_index=True)

    out_province = out_province.append({'province_name' : '天津市' , 'province_zip_code' : 120000, 'update_time': None, 'update_date': datetime.date(2020, 1, 22), 
                                        'province_confirmed': 4, 'province_suspected': None, 'province_cured': 0, 'province_dead': 0} , ignore_index=True)
    out_province = out_province.append({'province_name' : '天津市' , 'province_zip_code' : 120000, 'update_time': None, 'update_date': datetime.date(2020, 1, 24), 
                                        'province_confirmed': 8, 'province_suspected': None, 'province_cured': 0, 'province_dead': 0} , ignore_index=True)
    out_province = out_province.append({'province_name' : '天津市' , 'province_zip_code' : 120000, 'update_time': None, 'update_date': datetime.date(2020, 1, 25), 
                                        'province_confirmed': 10, 'province_suspected': None, 'province_cured': 0, 'province_dead': 0} , ignore_index=True)

    out_province = out_province.append({'province_name' : '新疆维吾尔自治区' , 'province_zip_code' : 650000, 'update_time': None, 'update_date': datetime.date(2020, 1, 24), 
                                        'province_confirmed': 2, 'province_suspected': None, 'province_cured': 0, 'province_dead': 0} , ignore_index=True)

# Back
    
    out_province = out_province.append({'province_name' : '安徽省' , 'province_zip_code' : 340000, 'update_time': None, 'update_date': datetime.date(2020, 3, 7), 
                                        'province_confirmed': 990, 'province_suspected': None, 'province_cured': 981, 'province_dead': 6} , ignore_index=True)
    
    
    out_province = out_province.append({'province_name' : '甘肃省' , 'province_zip_code' : 620000, 'update_time': None, 'update_date': datetime.date(2020, 2, 14), 
                                            'province_confirmed': 90, 'province_suspected': None, 'province_cured': 44, 'province_dead': 2} , ignore_index=True)
    out_province = out_province.append({'province_name' : '甘肃省' , 'province_zip_code' : 620000, 'update_time': None, 'update_date': datetime.date(2020, 2, 22), 
                                            'province_confirmed': 91, 'province_suspected': None, 'province_cured': 76, 'province_dead': 2} , ignore_index=True)
    out_province = out_province.append({'province_name' : '甘肃省' , 'province_zip_code' : 620000, 'update_time': None, 'update_date': datetime.date(2020, 2, 25), 
                                            'province_confirmed': 91, 'province_suspected': None, 'province_cured': 80, 'province_dead': 2} , ignore_index=True)
    out_province = out_province.append({'province_name' : '甘肃省' , 'province_zip_code' : 620000, 'update_time': None, 'update_date': datetime.date(2020, 2, 27), 
                                            'province_confirmed': 91, 'province_suspected': None, 'province_cured': 82, 'province_dead': 2} , ignore_index=True)
    out_province = out_province.append({'province_name' : '甘肃省' , 'province_zip_code' : 620000, 'update_time': None, 'update_date': datetime.date(2020, 2, 29), 
                                            'province_confirmed': 91, 'province_suspected': None, 'province_cured': 82, 'province_dead': 2} , ignore_index=True)

    out_province.loc[(out_province.update_date == datetime.date(2020,1,31)) & (out_province.province_name == '贵州省'), 'province_confirmed'] = 15
    out_province.loc[(out_province.update_date == datetime.date(2020,1,31)) & (out_province.province_name == '贵州省'), 'province_cured'] = 1

    out_province = out_province.append({'province_name' : '贵州省' , 'province_zip_code' : 520000, 'update_time': None, 'update_date': datetime.date(2020, 2, 1), 
                                            'province_confirmed': 29, 'province_suspected': None, 'province_cured': 2, 'province_dead': 0} , ignore_index=True)

    out_province = out_province.append({'province_name' : '贵州省' , 'province_zip_code' : 520000, 'update_time': None, 'update_date': datetime.date(2020, 2, 24), 
                                            'province_confirmed': 146, 'province_suspected': None, 'province_cured': 103, 'province_dead': 2} , ignore_index=True)
    out_province = out_province.append({'province_name' : '贵州省' , 'province_zip_code' : 520000, 'update_time': None, 'update_date': datetime.date(2020, 2, 26), 
                                            'province_confirmed': 146, 'province_suspected': None, 'province_cured': 112, 'province_dead': 2} , ignore_index=True)
    out_province = out_province.append({'province_name' : '贵州省' , 'province_zip_code' : 520000, 'update_time': None, 'update_date': datetime.date(2020, 2, 28), 
                                            'province_confirmed': 146, 'province_suspected': None, 'province_cured': 112, 'province_dead': 2} , ignore_index=True)
    out_province = out_province.append({'province_name' : '贵州省' , 'province_zip_code' : 520000, 'update_time': None, 'update_date': datetime.date(2020, 2, 29), 
                                            'province_confirmed': 146, 'province_suspected': None, 'province_cured': 112, 'province_dead': 2} , ignore_index=True)
    out_province = out_province.append({'province_name' : '贵州省' , 'province_zip_code' : 520000, 'update_time': None, 'update_date': datetime.date(2020, 3, 1), 
                                            'province_confirmed': 146, 'province_suspected': None, 'province_cured': 114, 'province_dead': 2} , ignore_index=True)
    out_province = out_province.append({'province_name' : '贵州省' , 'province_zip_code' : 520000, 'update_time': None, 'update_date': datetime.date(2020, 3, 3), 
                                            'province_confirmed': 146, 'province_suspected': None, 'province_cured': 114, 'province_dead': 2} , ignore_index=True)
    out_province = out_province.append({'province_name' : '贵州省' , 'province_zip_code' : 520000, 'update_time': None, 'update_date': datetime.date(2020, 3, 4), 
                                            'province_confirmed': 146, 'province_suspected': None, 'province_cured': 114, 'province_dead': 2} , ignore_index=True)
    out_province = out_province.append({'province_name' : '贵州省' , 'province_zip_code' : 520000, 'update_time': None, 'update_date': datetime.date(2020, 3, 5), 
                                            'province_confirmed': 146, 'province_suspected': None, 'province_cured': 114, 'province_dead': 2} , ignore_index=True)
    out_province = out_province.append({'province_name' : '贵州省' , 'province_zip_code' : 520000, 'update_time': None, 'update_date': datetime.date(2020, 3, 6), 
                                            'province_confirmed': 146, 'province_suspected': None, 'province_cured': 114, 'province_dead': 2} , ignore_index=True)

    
    out_province = out_province.append({'province_name' : '海南省' , 'province_zip_code' : 460000, 'update_time': None, 'update_date': datetime.date(2020, 3, 2), 
                                            'province_confirmed': 168, 'province_suspected': None, 'province_cured': 151, 'province_dead': 5} , ignore_index=True)
    out_province = out_province.append({'province_name' : '海南省' , 'province_zip_code' : 460000, 'update_time': None, 'update_date': datetime.date(2020, 3, 6), 
                                            'province_confirmed': 168, 'province_suspected': None, 'province_cured': 158, 'province_dead': 6} , ignore_index=True)
    out_province = out_province.append({'province_name' : '海南省' , 'province_zip_code' : 460000, 'update_time': None, 'update_date': datetime.date(2020, 3, 7), 
                                            'province_confirmed': 168, 'province_suspected': None, 'province_cured': 158, 'province_dead': 6} , ignore_index=True)


    out_province = out_province.append({'province_name' : '河南省' , 'province_zip_code' : 410000, 'update_time': None, 'update_date': datetime.date(2020, 3, 9), 
                                        'province_confirmed': 1272, 'province_suspected': None, 'province_cured': 1247, 'province_dead': 22} , ignore_index=True)
    out_province = out_province.append({'province_name' : '河南省' , 'province_zip_code' : 410000, 'update_time': None, 'update_date': datetime.date(2020, 3, 10), 
                                        'province_confirmed': 1272, 'province_suspected': None, 'province_cured': 1249, 'province_dead': 22} , ignore_index=True)

    
    out_province = out_province.append({'province_name' : '内蒙古自治区' , 'province_zip_code' : 150000, 'update_time': None, 'update_date': datetime.date(2020, 2, 11), 
                                            'province_confirmed': 60, 'province_suspected': None, 'province_cured': 5, 'province_dead': 0} , ignore_index=True)

    out_province = out_province.append({'province_name' : '内蒙古自治区' , 'province_zip_code' : 150000, 'update_time': None, 'update_date': datetime.date(2020, 2, 25), 
                                            'province_confirmed': 75, 'province_suspected': None, 'province_cured': 35, 'province_dead': 0} , ignore_index=True)
    out_province = out_province.append({'province_name' : '内蒙古自治区' , 'province_zip_code' : 150000, 'update_time': None, 'update_date': datetime.date(2020, 3, 9), 
                                            'province_confirmed': 75, 'province_suspected': None, 'province_cured': 70, 'province_dead': 1} , ignore_index=True)
    out_province = out_province.append({'province_name' : '内蒙古自治区' , 'province_zip_code' : 150000, 'update_time': None, 'update_date': datetime.date(2020, 3, 10), 
                                            'province_confirmed': 75, 'province_suspected': None, 'province_cured': 70, 'province_dead': 1} , ignore_index=True)


    out_province = out_province.append({'province_name' : '吉林省' , 'province_zip_code' : 220000, 'update_time': None, 'update_date': datetime.date(2020, 1, 26), 
                                            'province_confirmed': 4, 'province_suspected': None, 'province_cured': 0, 'province_dead': 0} , ignore_index=True)
    out_province = out_province.append({'province_name' : '吉林省' , 'province_zip_code' : 220000, 'update_time': None, 'update_date': datetime.date(2020, 3, 3), 
                                            'province_confirmed': 93, 'province_suspected': None, 'province_cured': 84, 'province_dead': 1} , ignore_index=True)
    out_province = out_province.append({'province_name' : '吉林省' , 'province_zip_code' : 220000, 'update_time': None, 'update_date': datetime.date(2020, 3, 7), 
                                            'province_confirmed': 93, 'province_suspected': None, 'province_cured': 90, 'province_dead': 1} , ignore_index=True)
    out_province = out_province.append({'province_name' : '吉林省' , 'province_zip_code' : 220000, 'update_time': None, 'update_date': datetime.date(2020, 3, 8), 
                                            'province_confirmed': 93, 'province_suspected': None, 'province_cured': 90, 'province_dead': 1} , ignore_index=True)

    out_province = out_province.append({'province_name' : '辽宁省' , 'province_zip_code' : 210000, 'update_time': None, 'update_date': datetime.date(2020, 2, 28), 
                                            'province_confirmed': 121, 'province_suspected': None, 'province_cured': 95, 'province_dead': 1} , ignore_index=True)
    out_province = out_province.append({'province_name' : '辽宁省' , 'province_zip_code' : 210000, 'update_time': None, 'update_date': datetime.date(2020, 3, 4), 
                                            'province_confirmed': 125, 'province_suspected': None, 'province_cured': 106, 'province_dead': 1} , ignore_index=True)
    out_province = out_province.append({'province_name' : '辽宁省' , 'province_zip_code' : 210000, 'update_time': None, 'update_date': datetime.date(2020, 3, 5), 
                                            'province_confirmed': 125, 'province_suspected': None, 'province_cured': 106, 'province_dead': 1} , ignore_index=True)
    out_province = out_province.append({'province_name' : '辽宁省' , 'province_zip_code' : 210000, 'update_time': None, 'update_date': datetime.date(2020, 3, 6), 
                                            'province_confirmed': 125, 'province_suspected': None, 'province_cured': 106, 'province_dead': 1} , ignore_index=True)
    out_province = out_province.append({'province_name' : '辽宁省' , 'province_zip_code' : 210000, 'update_time': None, 'update_date': datetime.date(2020, 3, 9), 
                                            'province_confirmed': 125, 'province_suspected': None, 'province_cured': 110, 'province_dead': 1} , ignore_index=True)


    out_province = out_province.append({'province_name' : '宁夏回族自治区' , 'province_zip_code' : 640000, 'update_time': None, 'update_date': datetime.date(2020, 2, 5), 
                                            'province_confirmed': 34, 'province_suspected': None, 'province_cured': 1, 'province_dead': 0} , ignore_index=True)
    out_province = out_province.append({'province_name' : '宁夏回族自治区' , 'province_zip_code' : 640000, 'update_time': None, 'update_date': datetime.date(2020, 2, 16), 
                                            'province_confirmed': 70, 'province_suspected': None, 'province_cured': 33, 'province_dead': 0} , ignore_index=True)
    out_province = out_province.append({'province_name' : '宁夏回族自治区' , 'province_zip_code' : 640000, 'update_time': None, 'update_date': datetime.date(2020, 2, 22), 
                                            'province_confirmed': 71, 'province_suspected': None, 'province_cured': 48, 'province_dead': 0} , ignore_index=True)
    out_province = out_province.append({'province_name' : '宁夏回族自治区' , 'province_zip_code' : 640000, 'update_time': None, 'update_date': datetime.date(2020, 2, 28), 
                                            'province_confirmed': 73, 'province_suspected': None, 'province_cured': 68, 'province_dead': 0} , ignore_index=True)
    out_province = out_province.append({'province_name' : '宁夏回族自治区' , 'province_zip_code' : 640000, 'update_time': None, 'update_date': datetime.date(2020, 3, 1), 
                                            'province_confirmed': 74, 'province_suspected': None, 'province_cured': 69, 'province_dead': 0} , ignore_index=True)
    out_province = out_province.append({'province_name' : '宁夏回族自治区' , 'province_zip_code' : 640000, 'update_time': None, 'update_date': datetime.date(2020, 3, 3), 
                                            'province_confirmed': 75, 'province_suspected': None, 'province_cured': 69, 'province_dead': 0} , ignore_index=True)
    out_province = out_province.append({'province_name' : '宁夏回族自治区' , 'province_zip_code' : 640000, 'update_time': None, 'update_date': datetime.date(2020, 3, 5), 
                                            'province_confirmed': 75, 'province_suspected': None, 'province_cured': 69, 'province_dead': 0} , ignore_index=True)

    out_province = out_province.append({'province_name' : '青海省' , 'province_zip_code' : 630000, 'update_time': None, 'update_date': datetime.date(2020, 1, 26), 
                                            'province_confirmed': 4, 'province_suspected': None, 'province_cured': 0, 'province_dead': 0} , ignore_index=True)
    out_province = out_province.append({'province_name' : '青海省' , 'province_zip_code' : 630000, 'update_time': None, 'update_date': datetime.date(2020, 1, 28), 
                                            'province_confirmed': 6, 'province_suspected': None, 'province_cured': 0, 'province_dead': 0} , ignore_index=True)
    out_province = out_province.append({'province_name' : '青海省' , 'province_zip_code' : 630000, 'update_time': None, 'update_date': datetime.date(2020, 1, 29), 
                                            'province_confirmed': 6, 'province_suspected': None, 'province_cured': 0, 'province_dead': 0} , ignore_index=True)
    out_province = out_province.append({'province_name' : '青海省' , 'province_zip_code' : 630000, 'update_time': None, 'update_date': datetime.date(2020, 2, 7), 
                                            'province_confirmed': 18, 'province_suspected': None, 'province_cured': 3, 'province_dead': 0} , ignore_index=True)
    out_province = out_province.append({'province_name' : '青海省' , 'province_zip_code' : 630000, 'update_time': None, 'update_date': datetime.date(2020, 2, 8), 
                                            'province_confirmed': 18, 'province_suspected': None, 'province_cured': 3, 'province_dead': 0} , ignore_index=True)
    out_province = out_province.append({'province_name' : '青海省' , 'province_zip_code' : 630000, 'update_time': None, 'update_date': datetime.date(2020, 2, 9), 
                                            'province_confirmed': 18, 'province_suspected': None, 'province_cured': 3, 'province_dead': 0} , ignore_index=True)
    out_province = out_province.append({'province_name' : '青海省' , 'province_zip_code' : 630000, 'update_time': None, 'update_date': datetime.date(2020, 2, 10), 
                                            'province_confirmed': 18, 'province_suspected': None, 'province_cured': 3, 'province_dead': 0} , ignore_index=True)
    out_province = out_province.append({'province_name' : '青海省' , 'province_zip_code' : 630000, 'update_time': None, 'update_date': datetime.date(2020, 2, 14), 
                                            'province_confirmed': 18, 'province_suspected': None, 'province_cured': 11, 'province_dead': 0} , ignore_index=True)
    out_province = out_province.append({'province_name' : '青海省' , 'province_zip_code' : 630000, 'update_time': None, 'update_date': datetime.date(2020, 2, 16), 
                                            'province_confirmed': 18, 'province_suspected': None, 'province_cured': 13, 'province_dead': 0} , ignore_index=True)
    out_province = out_province.append({'province_name' : '青海省' , 'province_zip_code' : 630000, 'update_time': None, 'update_date': datetime.date(2020, 2, 17), 
                                            'province_confirmed': 18, 'province_suspected': None, 'province_cured': 13, 'province_dead': 0} , ignore_index=True)
    out_province = out_province.append({'province_name' : '青海省' , 'province_zip_code' : 630000, 'update_time': None, 'update_date': datetime.date(2020, 2, 20), 
                                            'province_confirmed': 18, 'province_suspected': None, 'province_cured': 16, 'province_dead': 0} , ignore_index=True)
    out_province = out_province.append({'province_name' : '青海省' , 'province_zip_code' : 630000, 'update_time': None, 'update_date': datetime.date(2020, 2, 22), 
                                            'province_confirmed': 18, 'province_suspected': None, 'province_cured': 18, 'province_dead': 0} , ignore_index=True)

    
    out_province = out_province.append({'province_name' : '陕西省' , 'province_zip_code' : 610000, 'update_time': None, 'update_date': datetime.date(2020, 3, 3), 
                                            'province_confirmed': 245, 'province_suspected': None, 'province_cured': 218, 'province_dead': 1} , ignore_index=True)
    out_province = out_province.append({'province_name' : '陕西省' , 'province_zip_code' : 610000, 'update_time': None, 'update_date': datetime.date(2020, 3, 7), 
                                            'province_confirmed': 245, 'province_suspected': None, 'province_cured': 226, 'province_dead': 1} , ignore_index=True)
    out_province = out_province.append({'province_name' : '陕西省' , 'province_zip_code' : 610000, 'update_time': None, 'update_date': datetime.date(2020, 3, 9), 
                                            'province_confirmed': 245, 'province_suspected': None, 'province_cured': 227, 'province_dead': 1} , ignore_index=True)
    
    out_province = out_province.append({'province_name' : '山西省' , 'province_zip_code' : 140000, 'update_time': None, 'update_date': datetime.date(2020, 3, 6), 
                                            'province_confirmed': 133, 'province_suspected': None, 'province_cured': 126, 'province_dead': 0} , ignore_index=True)
    out_province = out_province.append({'province_name' : '山西省' , 'province_zip_code' : 140000, 'update_time': None, 'update_date': datetime.date(2020, 3, 7), 
                                            'province_confirmed': 133, 'province_suspected': None, 'province_cured': 126, 'province_dead': 0} , ignore_index=True)
    out_province = out_province.append({'province_name' : '山西省' , 'province_zip_code' : 140000, 'update_time': None, 'update_date': datetime.date(2020, 3, 8), 
                                            'province_confirmed': 133, 'province_suspected': None, 'province_cured': 126, 'province_dead': 0} , ignore_index=True)
    
    out_province = out_province.append({'province_name' : '天津市' , 'province_zip_code' : 120000, 'update_time': None, 'update_date': datetime.date(2020, 3, 2), 
                                            'province_confirmed': 136, 'province_suspected': None, 'province_cured': 118, 'province_dead': 3} , ignore_index=True)
    out_province = out_province.append({'province_name' : '天津市' , 'province_zip_code' : 120000, 'update_time': None, 'update_date': datetime.date(2020, 3, 4), 
                                            'province_confirmed': 136, 'province_suspected': None, 'province_cured': 128, 'province_dead': 3} , ignore_index=True)
    out_province = out_province.append({'province_name' : '天津市' , 'province_zip_code' : 120000, 'update_time': None, 'update_date': datetime.date(2020, 3, 6), 
                                            'province_confirmed': 136, 'province_suspected': None, 'province_cured': 128, 'province_dead': 3} , ignore_index=True)
    out_province = out_province.append({'province_name' : '天津市' , 'province_zip_code' : 120000, 'update_time': None, 'update_date': datetime.date(2020, 3, 7), 
                                            'province_confirmed': 136, 'province_suspected': None, 'province_cured': 128, 'province_dead': 3} , ignore_index=True)
    out_province = out_province.append({'province_name' : '天津市' , 'province_zip_code' : 120000, 'update_time': None, 'update_date': datetime.date(2020, 3, 8), 
                                            'province_confirmed': 136, 'province_suspected': None, 'province_cured': 129, 'province_dead': 3} , ignore_index=True)

    out_province = out_province.append({'province_name' : '新疆维吾尔自治区' , 'province_zip_code' : 650000, 'update_time': None, 'update_date': datetime.date(2020, 2, 25), 
                                            'province_confirmed': 76, 'province_suspected': None, 'province_cured': 30, 'province_dead': 2} , ignore_index=True)

    out_province = out_province.append({'province_name' : '云南省' , 'province_zip_code' : 530000, 'update_time': None, 'update_date': datetime.date(2020, 3, 4), 
                                            'province_confirmed': 174, 'province_suspected': None, 'province_cured': 169, 'province_dead': 2} , ignore_index=True)
    out_province = out_province.append({'province_name' : '云南省' , 'province_zip_code' : 530000, 'update_time': None, 'update_date': datetime.date(2020, 3, 5), 
                                            'province_confirmed': 174, 'province_suspected': None, 'province_cured': 169, 'province_dead': 2} , ignore_index=True)
